In [1]:
import sys

sys.path.append("lib/")

import pandas as pd
import geopandas as gpd
import folium
import os
import glob

import TFilter as f
from TDataFrame import TDataFrame
from TDataType import TDataType as tdt
from TEvent import TEvent

In [2]:
# load data
filepaths = glob.glob(os.path.join("../../../DP/filtered/", "*.csv"))

# Loaded TDFs
ltdfs = []

LIMIT = 20
i = 0

for filename in filepaths:
    df = pd.read_csv(filename, sep=";", encoding="latin-1", low_memory=False)
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.IS_Longitude, df.IS_Latitude),
        crs="EPSG:4326",
    )

    # add to list
    ltdfs.append(TDataFrame(filename, gdf))

    i += 1
    if i > LIMIT:
        break

len(ltdfs)

21

In [3]:
# settings
T_Line = ("*", "*", "*")
T_Date = ("01.10.2022", "31.10.2022")
T_Time = ("00:00:00", "23:59:59")
T_Vehicle = "*"

T_Data_type = tdt.EMB_1

T_Map_events = True
T_Depo = {"c": (48.179637, 17.162346), "d": 250}


# filter all ltdfs and data
data = f.filter_tdfs(ltdfs, T_Line, T_Date, T_Vehicle)
for d in data:
    d.filter_rows(T_Data_type, T_Time)

print(len(data))

# detect events
events = f.detect_events(data)
events = f.filter_depo(events, T_Depo)

print(len(events))

21
58


In [9]:
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

# plot into map
m = folium.Map(prefer_canvas=True)

locations = []
popups = []
n = 0

# extract data into arrays
for d in data:
    locations += d.get_lat_lons()
    popups += d.get_popups()
    n += len(d.rows_filtered)

# plot data
if not T_Map_events:
    HeatMap(locations, name="HeatMap").add_to(m)
    MarkerCluster(locations, popups, name="Cluster").add_to(m)
else:
    # Create a MarkerCluster for events
    event_cluster = MarkerCluster()
    for event in events:
        popup_content = f"Start: {event.start_time.strftime('%H:%M:%S')}, End: {event.end_time.strftime('%H:%M:%S')}"
        event_cluster.add_child(
            folium.Marker(
                location=(event.points[0][0], event.points[0][1]), popup=popup_content
            )
        )
    m.add_child(event_cluster)

    # Extract latitude and longitude for heatmap
    heatmap_data = [[point[0], point[1]] for event in events for point in event.points]
    HeatMap(heatmap_data).add_to(m)

    # Add a circle representing the filtered-out area
    folium.Circle(
        location=T_Depo["c"],
        radius=T_Depo["d"],
        color="transparent",
        fill=True,
        fill_color="red",
        fill_opacity=0.2,
    ).add_to(m)


# set the zoom to the maximum possible
m.fit_bounds(m.get_bounds())

# layer control
folium.LayerControl().add_to(m)

m.save("map.html")

print(n)
m

613


In [5]:
# co dalej?
#
# asi by sa hodilo, aby sme vedeli odlisit tie miesta brzdenia, ktore su naozaj na tom mieste na ktorom sa stali, nie na doplnenom
# Folium plugin HeatMapWithTime ?
# skusit polohu brzdeni interpolovat a nie len doplnovat? - stale nevyriesi problem ze su tie brzdenia vsade
#
# Event popup by mohol hovorit viac o tom evente - extract do TEvent modelu
# hodil by sa refaktor vsetkeho tu, je to pain s tym pracovat